In [9]:
import scipy.io as sio 
import numpy as np
import matplotlib.pyplot as plt
import show_image
from scipy import stats

##
# load dataset for boys and girls
# dividing into 5 equally split datasets for cross validation
# choosing five fold split as 10 results in highly uneven splits
load1 = sio.loadmat('boys.mat')
boys_data = load1['data'].astype(float)
load2 = sio.loadmat('girls.mat')
girls_data = load2['data'].astype(float) 

boys_num = boys_data.shape[1]
girls_num = girls_data.shape[1]

print boys_data.shape, girls_data.shape


(4225L, 116L) (4225L, 63L)


In [17]:
X = np.hstack((boys_data, girls_data))

y = np.hstack((np.ones((1,boys_num)), np.zeros((1,girls_num))))

d = X.shape[0]
m = X.shape[1]

print X.shape, y.shape

print (y == 0).sum(axis=1) / (1.0*m)

(4225L, 179L) (1L, 179L)
[ 0.35195531]


In [20]:
# naive bayes
cno = 2

# training, learn those parameters in the class conditional. 
# we have also learned the prior.

# estimate prior
py = np.zeros(cno)
for c in range(cno):
    py[c] = (y == c).sum() / (1.0*m)


In [19]:
print py

[ 0.35195531  0.64804469]


In [21]:

mu_mat = np.zeros((d, cno))
sig_mat = np.zeros((d, cno))
for i in range(d): 
    for c in range(cno): 
        mu_mat[i, c] = X[i, (y==c).flatten()].mean()
        sig_mat[i, c] = X[i, (y==c).flatten()].std()

In [23]:
print mu_mat
print sig_mat

[[ 142.42857143  147.93965517]
 [ 142.42857143  147.93965517]
 [ 142.41269841  147.93965517]
 ..., 
 [ 125.19047619  116.03448276]
 [ 125.77777778  114.72413793]
 [ 125.95238095  115.3362069 ]]
[[ 19.98309944  23.91302083]
 [ 19.98309944  23.91302083]
 [ 19.95362474  23.91302083]
 ..., 
 [ 69.67449424  66.75326817]
 [ 72.93276844  68.02415725]
 [ 75.33363434  69.67312151]]


In [36]:
# making predictions for each data point in X; 
predy = np.zeros(m)

for j in range(m):
    
    py_x = np.array([py[0], py[1]])
    for c in range(cno):
        for i in range(d):
            py_x[c] *= np.exp(-(X[i,j] - mu_mat[i,c])**2 / (2*sig_mat[i,c]**2)) / sig_mat[i,c]
            
    #print py_x
    
    predy[j] = np.argmax(py_x)
    
print (predy == 0).sum() / (1.0*m)    
print (predy == 1).sum() / (1.0*m)

1.0
0.0


In [41]:
# making predictions for each data point in X; 
predy = np.zeros(m)

for j in range(m):
    
    log_py_x = np.array([np.log(py[0]), np.log(py[1])])
    for c in range(cno):
        for i in range(d):
            log_py_x[c] += -(X[i,j] - mu_mat[i,c])**2 / (2*sig_mat[i,c]**2) - np.log(sig_mat[i,c])
            
    # print log_py_x
    
    predy[j] = np.argmax(log_py_x)
    
# print predy    
    
print (predy == 0).sum() / (1.0*m)    
print (predy == 1).sum() / (1.0*m)   


print np.sum(np.abs(predy - y)) / (1.0*m)

0.335195530726
0.664804469274
0.162011173184


In [26]:
print py

[ 0.35195531  0.64804469]


In [61]:
theta = 0.001 * np.random.randn(d,1)

T = 1000

eta = 0.0000001

for t in range(T): 
    theta_x = np.dot(theta.T, X)
    w = np.exp(-theta_x) / (1 + np.exp(-theta_x))
    
    #print w.shape
    #print y.shape
    
    tmp = (y-1) + w
    #print tmp.shape
    
    gradient = np.dot(X, tmp.T)
    
    #print gradient.shape
    
    #print gradient
    
    theta += eta * gradient

In [65]:
predy = np.zeros(m)
for j in range(m): 
    #print X[:,j].shape
    #print theta.shape
    py1x = 1 / (1 + np.exp(-np.dot(theta.flatten(), X[:,j].flatten())))
    if (py1x > 0.5):
        predy[j] = 1
    else:
        predy[j] = 0
        
print (predy == 0).sum() / (1.0*m)    
print (predy == 1).sum() / (1.0*m)   


print np.sum(np.abs(predy - y)) / (1.0*m)    

0.351955307263
0.648044692737
0.0


In [63]:
print predy

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [64]:
print y

[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
